# Basic Image Classification w/ Keras

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Classify images of Clothing

## Dataset Fetch & Load

In [30]:
fashion_minst = tf.keras.datasets.fashion_mnist

In [31]:
(train_imgs, train_labels), (test_imgs, test_labels) = fashion_minst.load_data()

In [32]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

### Training Dataset - EDA

In [48]:
train_imgs.shape

(60000, 28, 28)

In [49]:
set(train_labels)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

### Test Dataset - EDA

In [54]:
test_imgs.shape

(10000, 28, 28)

In [55]:
set(test_labels)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

## Dataset Preprocessing